# Tensorflow Object Detection Inference


In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 33kB/s 
     |████████████████████████████████| 460kB 49.3MB/s 
     |████████████████████████████████| 3.0MB 50.8MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


## Install Object Detection API

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2455, done.
remote: Counting objects: 100% (2455/2455), done.
remote: Compressing objects: 100% (2053/2053), done.
remote: Total 2455 (delta 599), reused 1212 (delta 374), pack-reused 0
Receiving objects: 100% (2455/2455), 30.85 MiB | 29.69 MiB/s, done.
Resolving deltas: 100% (599/599), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1628562 sha256=0a970fec65670373efe9f764b94f4371358b85c369de45a21d480c45e93b7f53
  Stored in directory: /tmp/pip-ephem-wheel-cache-jgx4y8aa/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=f258ad4bcfb88340c1532fd0e38ab69e798405195589159ff16de0028981313e
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=cd01fee961cf00bee3179e754ea2b06130cb56bf23ab5d40911616b8e9f30748
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=291b8f4a8e3a3808cf84d1cd4dffba7aff08b1bb23e9d0be0ca9

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


## Imports and setup

In [ ]:
import os
import io
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import time

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

Instructions for updating:
non-resource variables are not supported in the long term


## Connect with drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load model

In [ ]:
ROOT = 'gdrive/MyDrive/flores/MIA1'
PATH_TO_FROZEN_GRAPH = os.path.join(ROOT, 'models', 'saved_model', 'segmentation', 'model-6569', 'frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join(ROOT, 'flores_label_map.pbtxt')

print('Model path {}'.format(PATH_TO_FROZEN_GRAPH))
print('Label path {}'.format(PATH_TO_LABELS))

Model path gdrive/MyDrive/flores/MIA1/models/saved_model/segmentation/model-6569/frozen_inference_graph.pb
Label path gdrive/MyDrive/flores/MIA1/flores_label_map.pbtxt


In [ ]:
# Load saved model and build the detection function
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
print('Model loaded')

Model loaded


## Load label map

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'target'}}

## Run detections

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  return np.array(Image.open(path))

In [ ]:
IMAGES_FOLDER = os.path.join(ROOT, 'data', 'segmentation', 'images')
images_list = os.listdir(IMAGES_FOLDER)
target_folder = os.path.join(IMAGES_FOLDER, 'gen_masks')
print('Images folder: {}'.format(IMAGES_FOLDER))
print('Found: {}'.format(len(images_list)))

Images folder: gdrive/MyDrive/flores/MIA1/data/segmentation/images
Found: 100


In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
import time
for idx, image_name in enumerate(images_list):
  print('{}/{}. Running inference for {}'.format(idx + 1, len(images_list), image_name))
  image_np = load_image_into_numpy_array(os.path.join(IMAGES_FOLDER, image_name))

  # run detections
  start_ = time.time()
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  end_ = time.time()
  print('Tiempo', end_ - start_)
    
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      max_boxes_to_draw=1,
      min_score_thresh=.03,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  # plt.figure(figsize=(16, 20))
  # plt.imshow(image_np_with_detections)
  # analyze only one image
  img_detection = Image.fromarray(image_np_with_detections)
  img_detection.save(os.path.join(ROOT, 'data', 'segmentation', image_name)) 
  # break
plt.show()

1/100. Running inference for target_0.jpg
Instructions for updating:
box_ind is deprecated, use box_indices instead
Tiempo 39.65090584754944
2/100. Running inference for target_10.jpg
Tiempo 2.2007601261138916
3/100. Running inference for target_1.jpg
Tiempo 2.2582411766052246
4/100. Running inference for target_11.jpg
Tiempo 2.2469780445098877
5/100. Running inference for target_13.jpg
Tiempo 2.2604215145111084
6/100. Running inference for target_12.jpg
Tiempo 2.271757125854492
7/100. Running inference for target_14.jpg
Tiempo 2.267116069793701
8/100. Running inference for target_16.jpg
Tiempo 2.262579917907715
9/100. Running inference for target_15.jpg
Tiempo 2.331895112991333
10/100. Running inference for target_18.jpg
Tiempo 2.2990305423736572
11/100. Running inference for target_17.jpg
Tiempo 2.267305850982666
12/100. Running inference for target_19.jpg
Tiempo 2.2313172817230225
13/100. Running inference for target_2.jpg
Tiempo 2.2916758060455322
14/100. Running inference for targ